## Specification

The system must provide a function ``search``, with the following specification:
```
def search(query, ordering = 'normal', count = 10):
  ...
```

It `print`s out the results of the search, subject to the following rules:
1. It selects from the set of all recipes that contain __all__ of the words in the query (the positions/order of the words in the recipe are to be ignored).
2. It orders them based on the provided ordering (a string, meaning defined below).
3. It `print`s the top `count` matches only, preserving the order from best to worst. Must `print` just their title, one per line. Must be less than `count` if the search is specific enough that less than `count` recipes match.

As an aside, do not worry about memory usage. If duplicating some data can make your code faster/neater then feel free.



### Data set

A file, `recipes.json` is provided, containing 17K recipes. It can be parsed into a Python data structure using the [`json`](https://docs.python.org/3/library/json.html) module. It is a list, where each recipe is a dictionary containing various keys:
* `title` : Name of recipe; you can assume these are unique
* `categories` : A list of tags assigned to the recipe
* `ingredients` : What is in it, as a list
* `directions` : List of steps to make the recipe
* `rating` : A rating, out of 5, of how good it is
* `calories` : How many calories it has
* `protein` : How much protein is in it
* `fat` : How much fat is in it

Note that the data set was obtained via web scrapping and hence is noisy - every key except for `title` is missing from at least one recipe. Your code will need to cope with this.

You will probably want to explore the data before starting, so you have an idea of what your code has to deal with.

Data set came from https://www.kaggle.com/hugodarwood/epirecipes/version/2 though note it has been cleaned it up, by deleting duplicates and removing the really dodgy entries.



### Search

The search should check the following parts of the recipe (see data set description below):
* `title`
* `categories`
* `ingredients`
* `directions`

For instance, given the query "banana cheese" you would expect "Banana Layer Cake with Cream Cheese Frosting" in the results. Note that case is to be ignored ("banana" matches "Banana") and the words __do not__ have to be next to one another, in the same order as the search query or even in the same part of the recipe ("cheese" could appear in the title and "banana" in the ingredients). However, all words in the search query __must__ appear somewhere.



### Tokenisation

This is the term for breaking a sentence into each individual word (token). Traditionally done using regular expressions, and Python does have the `re` module, but there is no need to do that here (`re` can be quite fiddly). For matching words your tokenisation must follow the following steps:
1. Convert all punctuation and digits into spaces. For punctuation use the set in [`string.punctuation`](https://docs.python.org/3/library/string.html#string.punctuation), for digits [`string.digits`](https://docs.python.org/3/library/string.html#string.digits). You may find [`translate()`](https://docs.python.org/3/library/stdtypes.html#str.translate) interesting!
2. [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) to extract individual tokens.
3. Ignore any token that is less than $3$ characters long.
4. Make tokens lowercase.

When matching words for search (above) or ordering (below) it's only a match if you match an entire token. There are many scenarios where this simple approach will fail, but it's good enough for this exercise. The auto marker will be checking the above is followed! When doing a search the code should ignore terms in the search string that fail the above requirements.



### Ordering

There are three ordering modes to select from, each indicated by passing a string to the `search` function:
* `normal` - Based simply on the number of times the search terms appear in the recipe. A score is calculated and the order is highest to lowest. The score sums the following terms (repeated words are counted multiple times, i.e. "cheese cheese cheese" is $3$ matches to "cheese"):
    * $8 \times$ Number of times a query word appears in the title
    * $4 \times$ Number of times a query word appears in the categories
    * $2 \times$ Number of times a query word appears in the ingredients
    * $1 \times$ Number of times a query word appears in the directions
    * The `rating` of the recipe (if not available assume $0$)

* `simple` - Tries to minimise the complexity of the recipe, for someone who is in a rush. Orders to minimise the number of ingredients multiplied by the numbers of steps in the directions.

* `healthy` - Order from lowest to highest by this cost function:
$$\frac{|\texttt{calories} - 510n|}{510} + 2\frac{|\texttt{protein} - 18n|}{18} + 4\frac{|\texttt{fat} - 150n|}{150}$$
Where $n \in \mathbb{N}^+$ is selected to minimise the cost ($n$ is a positive integer and $n=0$ is not allowed). This can be understood in terms of the numbers $510$, $18$ and $150$ being a third of the recommended daily intake (three meals per day) for an average person, and $n$ being the number of whole meals the person gets out of cooking/making the recipe. So this tries to select recipes that neatly divide into a set of meals that are the right amount to consume for a healthy, balanced diet. Try not to overthink the optimisation of $n$, as it's really quite simple to do!

To clarify the use of the ordering string, to get something healthy that contains cheese you might call `search('cheese', 'healthy')`. In the case of a recipe that is missing a key in its dictionary the rules are different for each search mode:
* `normal` - Consider a missing entry in the recipe (e.g. no `ingredients` are provided) to simply mean that entry can't match any search words (because it has none!), but the item is still eligible for inclusion in the results, assuming it can match the search with a different entry.
* `simple` - If a recipe is missing either `ingredients` or `directions` it is dropped from such a search result. Because the data is messy if either of these lists is of length $1$ it should be assumed that the list extraction has failed and the recipe is to also be dropped from the search results.
* `healthy` - If any of `calories`, `protein` or `fat` is missing the recipe should be dropped from the result.



### Extra

You may find the [_inverted index_](https://en.wikipedia.org/wiki/Inverted_index) interesting. It's a data structure used by search engines. For each word a user may search for this contains a list of all documents (recipes) that contain the word. This may take a little effort to understand, but the resulting code will be both faster and neater.

### Importing the necessary libraries

In [1]:
import operator
import json
import string
import time
import numpy as np
from functools import reduce

### Opening and reading the json file

In [2]:
with open('recipes.json') as json_file:
    data_recipes = json.load(json_file) 

### Functions used in the code

In [3]:
# Translators to modify punctuation and digits(make them whitespaces)          
translator_pun=str.maketrans(string.punctuation, ' '*len(string.punctuation))
translator_dig=str.maketrans(string.digits, ' '*len(string.digits))

    
def tokenisation_fun(name, i):
    """This function takes as inputs a key of the dictionary with all recipes,
       and the index of the entry and tokenize its value and returns it."""
    r = data_recipes[i][name]
    r = ' '.join(item for item in r)
    translator_pun=str.maketrans(string.punctuation, ' '*len(string.punctuation))
    translator_dig=str.maketrans(string.digits, ' '*len(string.digits))
    r = r.translate(translator_pun).translate(translator_dig).lower().split()
    r = [item for item in r if len(item) >= 3]
    return r

def counter_fun(word, text):
    """This function takes as input a specific word and a string
       and returns the number of times the word exists on the string."""
    
    return text.count(word)

def normal_fun(word,i):
    """This function takes as input a word and an index number and calculates a score regarding
       to how many times a word exists in the title, category, ingredient and direction of each entry. 
       Then it returns the score."""
    
    score_t = (8 * counter_fun(word,titles[i]))
    score_c = (4 * counter_fun(word,categories[i]))
    score_i = (2 * counter_fun(word,ingredients[i]))
    score_d = (1 * counter_fun(word,directions[i]))     
    score = score_t + score_c + score_i + score_d  
    return score
      

def simple_fun(directions, ingredients):
    """This function takes as inputs the directions and ingredients of the current entry
       and calculates a score according to the length of the list with directions and
       ingredients and returns it. The score is the multiplication of the lengths of the lists."""
    
    x = len(directions)
    y = len(ingredients)
    if x > 1 and y > 1:
        return x * y
    
    
def healthy_fun(calories,protein,fat):
    """This function takes as inputs the calories, protein and fat of the current entry.
       The positive integer 𝑛 is selected to minimise the cost_function below. It returns
       the score which minimizes the cost_funtion."""
    best_cost =  np.inf
    for n in range(1,5):
        cost_function = (abs(calories -510*n)/510) + 2 * (abs(protein -18 *n)/18) + 4 * (abs(fat-150*n)/150) 
        
        if cost_function < best_cost:
            best_cost = cost_function
    return best_cost
       
    
def common_indices_fun(query):
    """This function takes as input the query and returns a list with all common indices
       where the words in the query belong."""
    new_list = []
    for word in query:
        if word in words_dict.keys():
            new_list.append(words_dict[word])
        else:
            return None
          
        a = list(reduce(set.intersection, [set(item) for item in new_list ]))
    return a
    

In [4]:
check_list = [] # a list of lists, in which each list contains the tokenised words for each entry 
all_words_list = [] # a list with all the tokenised words from the data
titles = [] # a list of lists, in which each list contains the tokenised title's words for each entry
categories = [] # a list of lists, in which each list contains the tokenised category's words for each entry
ingredients = [] # a list of lists, in which each list contains the tokenised ingredient's words for each entry
directions = [] # a list of lists, in which each list contains the tokenised direction's words for each entry

# Looping through recipes in our data_recipes and calling the tokenisation_fun to tokenize the words on each entry
for i, recipe in enumerate(data_recipes):
    
    lst = []
    title = recipe['title'].translate(translator_pun).translate(translator_dig).lower().split()
    titles_list = [ t for t in title if len(t) >= 3]
    
    # Initially we check whether the specific key exists on each entry and then we tokenize it, using always the entry's index
    if 'categories' in recipe.keys(): 
        categories_list = tokenisation_fun('categories',i)
 
    if 'ingredients' in recipe.keys(): 
        ingredients_list = tokenisation_fun('ingredients',i)

    if 'directions' in recipe.keys():
        directions_list = tokenisation_fun('directions',i)


    lst = titles_list + categories_list + ingredients_list + directions_list
    check_list.append(lst)
    all_words_list += lst
    titles.append(titles_list)
    categories.append(categories_list)
    ingredients.append(ingredients_list)
    directions.append(directions_list)

In [5]:
# Now we create a set with all tokenised words. A set contains only unique values, so we just have the unique words that
# the data is using.
unique_items = set(all_words_list)

In [6]:
# The dictionary words_dict is a dictionary with key-value pairs where the key is the word and its value is a list
# of the indices of entries on which the specific word occurs.

words_dict = {}

for word in unique_items:
    lst_words = []
    for i in range(len(check_list)):
        if word in check_list[i]:
            lst_words.append(i)
    words_dict.update({word : lst_words})                     

### Search function

In [7]:
def search(query, ordering = 'normal', count = 10):
    """This function takes as input a query which is a string of words, a definition of the ordering(whether
       is 'normal','simple','healthy') and a count number and returns as many recipes as the count number
       according to some score being calculated. More specifically, it selects from the set of all recipes
       that contain all of the words in the query, it orders them based on the provided ordering, and then 
       it prints the top count matches only, preserving the order from best to worst"""
    
    translator_pun=str.maketrans(string.punctuation, ' '*len(string.punctuation))
    translator_dig=str.maketrans(string.digits, ' '*len(string.digits))
    
    # Using the translators above we tokenize the query as well and then we get a list of all tokenised words in the query
    query = query.translate(translator_pun).translate(translator_dig).lower().split()
    query = [word for word in query if len(word) >= 3]
    

    indices = common_indices_fun(query)
    
    if indices == None:
        return 
    else:
        
        # Creating three dictionaries that contain as their keys the indices of the matched recipes and
        # as their values the calculated scores in each ordering definition.
        score_normal_dict = {}
        score_simple_dict = {}
        score_healthy_dict = {}

        if ordering == 'normal':
            
            for i in indices:
                normal_score = 0
                for word in query:
                    normal_score += normal_fun(word,i)

                if 'rating' in data_recipes[i]:
                    normal_score += data_recipes[i]['rating']
                else:
                    normal_score += 0

                score_normal_dict[i]=normal_score

            # Sorting the dictionary according to the score/values and using reverse we get the scores from highest
            # to lowest 
            sorted_normal_score = sorted(score_normal_dict.items(), key=operator.itemgetter(1), reverse = True)[:count]  
            
            # Finding the corresponding title for each score
            for i in sorted_normal_score:
                print(data_recipes[i[0]]['title'])

        # Doing exactly the same thing as above for the other ordering definitions but now we are not using reverse as
        # we want the order to be from lowest to highest. Then printing out the corresponding titles.
        elif ordering == 'simple':

            for i in indices:
                simple_score = simple_fun(data_recipes[i]['directions'],data_recipes[i]['ingredients'])
                if simple_score != None:
                    score_simple_dict[i]=simple_score

            sorted_simple_score = sorted(score_simple_dict.items(), key=operator.itemgetter(1))[:count]
            for i in sorted_simple_score:
                print(data_recipes[i[0]]['title'])

        elif ordering == 'healthy':

            for i in indices:
                if 'calories' in data_recipes[i].keys() and 'fat' in data_recipes[i].keys() and 'protein' in data_recipes[i].keys():
                    healthy_score = healthy_fun(data_recipes[i]['calories'],data_recipes[i]['protein'],data_recipes[i]['fat'])
                    score_healthy_dict[i]=healthy_score

            sorted_healthy_score = sorted(score_healthy_dict.items(), key=operator.itemgetter(1))[:count]
            for i in sorted_healthy_score:
                print(data_recipes[i[0]]['title'])

In [9]:
%time search('banana cheese')

Banana Layer Cake with Cream Cheese Frosting 
Banana Layer Cake with White Chocolate-Cream Cheese Frosting and Walnuts 
Banana-Pineapple Layer Cake with Cream Cheese Frosting 
Fresh Banana Layer Cake 
Peanut Butter Banana Cream Pie 
Banana Layer Cake 
Banana Coconut Crunch Cake 
Carrot-Banana Cake 
Persimmon Cake with Cream Cheese Icing 
Peanut Butter Cheesecake with Caramelized Banana Topping 
Wall time: 8 ms
